In [ ]:
import keras

In [ ]:
from tensorflow.keras.applications import nasnet

In [ ]:
import ssl
import re
from PIL import Image

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [1]:
from load_dataset import get_available_moires, get_train_data

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
IMG_SIZE = (150,150)

# Daten laden

In [3]:
df = get_available_moires()

In [4]:
data, labels = get_train_data( df, IMG_SIZE )

 72%|█████████████████████████████▋           | 196/271 [12:48<00:58,  1.27it/s]/Users/frederic.birwe/Library/Python/3.10/lib/python/site-packages/PIL/Image.py:3182: DecompressionBombWarning: Image size (127715826 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|█████████████████████████████████████████| 271/271 [14:36<00:00,  3.23s/it]


In [ ]:
0 / 0

In [ ]:
train_datagen = ImageDataGenerator(
    vertical_flip=True,
    #preprocessing_function=randomly_reorder_channels,
    validation_split=0.2,
    #channel_shift_range=0.5
)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

# Modell laden

In [ ]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False
)
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()]
)

In [ ]:
model.fit(X_train, y_train, epochs=5)